In [ ]:
!pip install elasticsearch

In [16]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import pandas as pd
import json

import numpy as np

LOCAL = False

es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch"])

df_billboard = pd.read_json("./data/scrap200.json")

In [17]:
df_billboard.head()

,album,artist,rank
0,Please Excuse Me For Being Antisocial,Roddy Ricch,1
1,Hollywood's Bleeding,Post Malone,2
2,Fine Line,Harry Styles,3
3,JACKBOYS,JACKBOYS,4
4,Frozen II,Soundtrack,5


In [18]:
documents = df_billboard.fillna("").to_dict(orient="records")

In [19]:
documents[0:5]

[{'album': 'Please Excuse Me For Being Antisocial',
  'artist': 'Roddy Ricch',
  'rank': 1},
 {'album': "Hollywood's Bleeding", 'artist': 'Post Malone', 'rank': 2},
 {'album': 'Fine Line', 'artist': 'Harry Styles', 'rank': 3},
 {'album': 'JACKBOYS', 'artist': 'JACKBOYS', 'rank': 4},
 {'album': 'Frozen II', 'artist': 'Soundtrack', 'rank': 5}]

In [24]:
def generate_data(documents):
    for docu in documents:
        yield {
            "_index": "albums",
            "_type": "album",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es_client, generate_data(documents))

(200, [])

In [25]:
#!curl http://localhost:9200/_cat/indices?v

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2018 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" CONTENT="text/html; charset=utf-8">
<title>ERROR: The requested URL could not be retrieved</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2015 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ license and includes
 * contributions from numerous individuals and organizations.
 * Please see the COPYING and CONTRIBUTORS files for details.
 */

/*
 Stylesheet for Squid Error pages
 Adapted from design by Free CSS Templates
 http://www.freecsstemplates.org
 Released for free under a Creative Commons Attribution 2.5 License
*/

/* Page basics */
* {
	font-family: verdana, sans-serif;
}

html body {
	margin: 0;
	padding: 0;
	background: #efefef;
	font-size: 12px;
	color: #1e1e1e;
}

/* Page disp

In [26]:
#!curl http://localhost:9200/albums/_search

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2018 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" CONTENT="text/html; charset=utf-8">
<title>ERROR: The requested URL could not be retrieved</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2015 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ license and includes
 * contributions from numerous individuals and organizations.
 * Please see the COPYING and CONTRIBUTORS files for details.
 */

/*
 Stylesheet for Squid Error pages
 Adapted from design by Free CSS Templates
 http://www.freecsstemplates.org
 Released for free under a Creative Commons Attribution 2.5 License
*/

/* Page basics */
* {
	font-family: verdana, sans-serif;
}

html body {
	margin: 0;
	padding: 0;
	background: #efefef;
	font-size: 12px;
	color: #1e1e1e;
}

/* Page disp

In [38]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "Malone",
      "fields": [ "artist" ] 
    }
  }
}

In [39]:
result = es_client.search(index="albums", body=QUERY)
[elt['_source']['album'] for elt in result["hits"]["hits"]]

["Hollywood's Bleeding", 'beerbongs & bentleys', 'Stoney']